In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
from itertools import product
from sklearn.preprocessing import OneHotEncoder
import time
warnings.filterwarnings("ignore")

2025-08-12 15:27:12.025619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755012432.280520      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755012432.356825      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_df = pd.read_csv("/kaggle/input/preprocessed-dataset/preprocessed_train.csv")
# test_df = pd.read_csv("/kaggle/input/preprocessed-dataset/preprocessed_test.csv")

In [3]:
seq_ids = train_df["sequence_id"].values
seq_counters = train_df["sequence_counter"].values
seq_starts = np.array(train_df[train_df["sequence_counter"]==0].index)
seq_lengths = np.diff(np.concatenate([seq_starts, np.array([len(train_df)])]), n=1)

bins = np.arange(0, 300, 5)
plt.hist(seq_lengths, bins=bins)
plt.show()

In [4]:
print(f"Quantile {0.995*100}%: {np.quantile(seq_lengths, 0.995)}")
print(f"Max length: {seq_lengths.max()}")

Quantile 99.5%: 246.0
Max length: 700


In [5]:
len(seq_lengths) - int(len(seq_lengths) * 0.995)

38

In [6]:
train_df["sequence_type"].unique()

array(['Target', 'Non-Target'], dtype=object)

In [7]:
type_mapping = {"Target": 1, "Non-Target": 0}
train_df["sequence_type"] = train_df["sequence_type"].map(type_mapping)

In [8]:
gestures = pd.get_dummies(train_df["gesture"])
train_df = pd.concat([train_df, gestures], axis=1)
train_df = train_df.drop(["gesture"], axis=1)

In [9]:
gesture_mapping = {gesture: i for i, gesture in enumerate(gestures)}

In [20]:
from sklearn.model_selection import train_test_split
tf.random.set_seed(42)
acc_columns = ["acc_x", "acc_y", "acc_z", "acc_r", "acc_theta", "acc_phi"]
rot_columns = ["rot_w", "rot_x", "rot_y", "rot_z", "rot_omega", "rot_u_theta", "rot_u_phi"]
thm_columns = ["thm_1", "thm_2", "thm_3", "thm_4", "thm_5", "thm_mean"]

i_list = range(1, 6)
j_list = range(64)
comb = list(product(i_list, j_list))
tof_columns = [f"tof_{i}_v{j}" for i, j in comb]

def create_dataset(df, window_size=50, shift=25, batch_size=16, mode="train", train_size=0.8):
    print(">>>>> Start creating dataset...")
    start_time = time.time()
    
    seq_starts = np.array(df[df["sequence_counter"]==0].index)
    seq_lengths = np.diff(np.concatenate([seq_starts, np.array([len(df)])]), n=1)
    
    images = df[tof_columns].to_numpy()   # (samples, 5*64)
    images = images.reshape(-1, 64, 5)
    images = images.reshape(-1, 8, 8, 5)
    images = tf.convert_to_tensor(images, dtype=tf.float32)

    others = df[acc_columns + rot_columns + thm_columns].to_numpy()
    others = tf.convert_to_tensor(others, dtype=tf.float32)

    feature_columns = ['acc_x_mean', 'acc_y_mean', 'acc_z_mean', "acc_x_std", "acc_y_std", "acc_z_std",
                       'thm_1_std', 'thm_2_std', 'thm_3_std', 'thm_4_std', 'thm_5_std',
                       'adult_child', 'age', 'sex', 'height_cm',
                       'shoulder_to_wrist_cm', 'elbow_to_wrist_cm']
    features = df[feature_columns].to_numpy()
    features = tf.convert_to_tensor(features, dtype=tf.float32)

    if mode != "test":
        sequence_types = df[["sequence_type"]].values
        gestures = df[gesture_mapping.keys()].values
        targets = np.concatenate([sequence_types, gestures], axis=1)
        targets = tf.convert_to_tensor(targets, dtype=tf.float32)
        
        ids = np.arange(len(targets))
        seq_starts_train, seq_starts_valid, seq_lengths_train, seq_lengths_valid = train_test_split(seq_starts, seq_lengths, train_size=train_size, stratify=tf.argmax(tf.gather(targets[:, 1:], seq_starts), axis=-1), random_state=42)
        seq_ends_train = seq_starts_train + seq_lengths_train
        inputs = (images, others, features, targets)
        
        threshold = seq_lengths_train.min()
        inputs_train, targets_train = time_series_crop_window(inputs, seq_ends_train, 
                                                              seq_lengths_train, window_size=window_size,
                                                              shift=shift, threshold=threshold)
        valid_max_length = seq_lengths_valid.max()
        inputs_valid, targets_valid = create_window(inputs, seq_starts_valid, 
                                                    seq_lengths_valid, valid_max_length, "valid")
        
        type_train_ds = tf.data.Dataset.from_tensor_slices((inputs_train, targets_train[:, :1]))
        type_train_ds = type_train_ds.shuffle(len(inputs_train)).batch(batch_size)
        gesture_train_ds = tf.data.Dataset.from_tensor_slices((inputs_train, targets_train[:, 1:]))
        gesture_train_ds = gesture_train_ds.shuffle(len(inputs_train)).batch(batch_size)

        type_valid_ds = tf.data.Dataset.from_tensor_slices((inputs_valid, targets_valid[:, :1]))
        type_valid_ds = type_valid_ds.shuffle(len(inputs_valid)).batch(batch_size)
        gesture_valid_ds = tf.data.Dataset.from_tensor_slices((inputs_valid, targets_valid[:, 1:]))
        gesture_valid_ds = gesture_valid_ds.shuffle(len(inputs_valid)).batch(batch_size)

        end_time = time.time()
        diff_time = end_time - start_time
        print(">>>>> End creating dataset")
        print_converted_time(diff_time)
        return type_train_ds, type_valid_ds, gesture_train_ds, gesture_valid_ds
    else:
        inputs = (images, others, features)
        test_max_length = seq_length.max()
        inputs = create_window(inputs, seq_starts, seq_lengths, test_max_length, mode=mode)
        test_ds = tf.data.Dataset.from_tensor_slices(inputs)
        test_ds = test_ds.batch(batch_size)
        end_time = time.time()
        time_diff = end_time - start_time
        print(">>>>> End creating dataset")
        print_converted_time(diff_time)
    return test_ds


def print_converted_time(diff_time):
    hours, rem = divmod(diff_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print(f"Elapsed time: {int(hours)} hours {int(minutes)} minutes {seconds:.2f} seconds")


def create_window(inputs, seq_starts, seq_lengths, max_length, mode="valid"):
    images = inputs[0]
    others = inputs[1]
    features = inputs[2]

    image_windows = []
    other_windows = []
    feature_windows = []

    if mode != "test":
        targets = inputs[3]
        target_windows = []
    
    for start, length in zip(seq_starts, seq_lengths):
        image_window = images[start:start+length, :, :, :]
        other_window = others[start:start+length, :]
        feature_window = features[start]
        if mode != "test":
            target_window = targets[start]
        if length < max_length:
            image_window = tf.pad(image_window, 
                                  tf.constant([[0, max_length-int(length)],
                                               [0, 0],
                                               [0, 0],
                                               [0, 0]]), constant_values=0.0)
            other_window = tf.pad(other_window, 
                                  tf.constant([[0, max_length-int(length)],
                                               [0, 0]]), constant_values=0.0)
        image_windows.append(image_window)
        other_windows.append(other_window)
        feature_windows.append(feature_window)
        if mode != "test":
            target_windows.append(target_window)

    image_windows = tf.stack(image_windows)
    other_windows = tf.stack(other_windows)
    feature_windows = tf.stack(feature_windows)

    if mode != "test":
        target_windows = tf.stack(target_windows)
        return (image_windows, other_windows, feature_windows), target_windows
    else:
        return (image_windows, other_windows, feature_windows)


def time_series_crop_window(inputs, seq_ends, seq_lengths, window_size=50, shift=25, threshold=10):
    images = inputs[0]
    others = inputs[1]
    features = inputs[2]
    targets = inputs[3]

    image_windows = []
    other_windows = []
    feature_windows = []
    target_windows = []
    
    for end, length in zip(seq_ends, seq_lengths):
        num_windows = (length - window_size) // shift + 1
        rem = length - shift * num_windows
        window_sizes = [window_size for i in range(num_windows - 1)]
        window_sizes.append(rem)
        print(window_sizes)
        for i, size in enumerate(window_sizes):
            append_flag = False
            # image_window = images[start + i*shift: start + i*shift + size, :, :, :]
            # other_window = others[start + i*shift: start + i*shift + size, :]
            # feature_window = features[start + i*shift]
            # target_window = targets[start + i*shift]
            image_window = images[end - i*shift - size: end - i*shift, :, :, :]
            other_window = others[end - i*shift - size: end - i*shift, :]
            if i == len(window_size) - 1:
                print(other_window[:, :2])
                break
            
            feature_window = features[end - i*shift - size]
            target_window = targets[end - i*shift - size]
                
            if size == window_size:
                append_flag = True
            elif size < window_size and size >= threshold:
                append_flag = True
                # image_window = tf.pad(image_window, 
                #                       tf.constant([[0, window_size-size],
                #                                    [0, 0],
                #                                    [0, 0],
                #                                    [0, 0]]), constant_values=0.0)
                # other_window = tf.pad(other_window, 
                #                       tf.constant([[0, window_size-size],
                #                                    [0, 0]]), constant_values=0.0)

                image_window = tf.pad(image_window, 
                                      tf.constant([[window_size-size, 0],
                                                   [0, 0],
                                                   [0, 0],
                                                   [0, 0]]), constant_values=0.0)
                other_window = tf.pad(other_window, 
                                      tf.constant([[window_size-size, 0],
                                                   [0, 0]]), constant_values=0.0)
            if append_flag:
                image_windows.append(image_window)
                other_windows.append(other_window)
                feature_windows.append(feature_window)
                target_windows.append(target_window)

    image_windows = tf.stack(image_windows)
    other_windows = tf.stack(other_windows)
    feature_windows = tf.stack(feature_windows)
    target_windows = tf.stack(target_windows)
    return (image_windows, other_windows, feature_windows), target_windows

    
def allocate_index(images, others, features, targets, seq_starts, seq_lengths, ids):
    images = tf.gather(images, ids)
    others = tf.gather(others, ids)
    features = tf.gather(features, ids)
    targets = tf.gather(targets, ids)
    seq_starts = tf.gather(seq_starts, ids)
    seq_lengths = tf.gather(seq_lengths, ids)
    return images, othres, features, targets, seq_starts, seq_lengths

In [18]:
create_dataset(train_df, batch_size=64)

>>>>> Start creating dataset...
[43]
[35]
[25]
[50, 50, 50, 50, 41]
[25]
[44]
[36]
[50, 29]
[43]
[36]
[45]
[35]
[42]
[49]
[47]
[47]
[50, 34]
[50, 50, 40]
[29]
[50, 39]
[44]
[50, 49]
[38]
[36]
[37]
[50, 50, 50, 41]
[50, 45]
[50, 50, 30]
[50, 50, 30]
[50, 43]
[31]
[39]
[42]
[50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 25]
[29]
[37]
[50, 50, 41]
[50, 50, 34]
[50, 43]
[50, 29]
[27]
[50, 33]
[49]
[27]
[50, 50, 31]
[50, 50, 50, 35]
[25]
[39]
[27]
[48]
[26]
[42]
[31]
[25]
[28]
[25]
[39]
[28]
[48]
[31]
[25]
[46]
[47]
[48]
[25]
[26]
[49]
[42]
[30]
[50, 50, 35]
[48]
[50, 26]
[50, 29]
[25]
[44]
[32]
[50, 50, 39]
[49]
[47]
[26]
[27]
[30]
[29]
[27]
[29]
[26]
[35]
[44]
[44]
[50, 50, 31]
[31]
[25]
[50, 39]
[40]
[50, 39]
[50, 50, 50, 50, 50, 50, 37]
[43]
[26]
[50, 31]
[48]
[25]
[49]
[43]
[50, 38]
[33]
[50, 50, 31]
[50, 38]
[46]
[44]
[39]
[26]
[26]
[50, 29]
[31]
[50, 50, 36]
[39]
[41]
[33]
[29]
[36]
[50, 50, 28]
[27]
[32]
[33]
[50, 50, 27]
[30

KeyboardInterrupt: 

In [14]:
type_train_ds, type_valid_ds, gesture_train_ds, gesture_valid_ds = create_dataset(train_df, batch_size=64)

>>>>> Start creating dataset...
[43]
[35]
[25]
[50, 50, 50, 50, 41]
[25]
[44]
[36]
[50, 29]
[43]
[36]
[45]
[35]
[42]
[49]
[47]
[47]
[50, 34]
[50, 50, 40]
[29]
[50, 39]
[44]
[50, 49]
[38]
[36]
[37]
[50, 50, 50, 41]
[50, 45]
[50, 50, 30]
[50, 50, 30]
[50, 43]
[31]
[39]
[42]
[50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 25]
[29]
[37]
[50, 50, 41]
[50, 50, 34]
[50, 43]
[50, 29]
[27]
[50, 33]
[49]
[27]
[50, 50, 31]
[50, 50, 50, 35]
[25]
[39]
[27]
[48]
[26]
[42]
[31]
[25]
[28]
[25]
[39]
[28]
[48]
[31]
[25]
[46]
[47]
[48]
[25]
[26]
[49]
[42]
[30]
[50, 50, 35]
[48]
[50, 26]
[50, 29]
[25]
[44]
[32]
[50, 50, 39]
[49]
[47]
[26]
[27]
[30]
[29]
[27]
[29]
[26]
[35]
[44]
[44]
[50, 50, 31]
[31]
[25]
[50, 39]
[40]
[50, 39]
[50, 50, 50, 50, 50, 50, 37]
[43]
[26]
[50, 31]
[48]
[25]
[49]
[43]
[50, 38]
[33]
[50, 50, 31]
[50, 38]
[46]
[44]
[39]
[26]
[26]
[50, 29]
[31]
[50, 50, 36]
[39]
[41]
[33]
[29]
[36]
[50, 50, 28]
[27]
[32]
[33]
[50, 50, 27]
[30

KeyboardInterrupt: 

In [ ]:
print(f"#batch: {len(type_train_ds)}")
for i, (inputs, targets) in enumerate(type_train_ds):
    if i == 0:
        print(inputs[0].shape)
        print(inputs[1].shape)
        print(inputs[2].shape)
        break

In [ ]:
def get_sequence_targets(df):
    seq_ids = df["sequence_id"].values
    seq_starts = np.array(df[df["sequence_counter"]==0].index)
    sequence_targets = df.loc[seq_starts, "sequence_type"].values
    return sequence_targets

In [ ]:
gesture_columns = gestures.columns
def get_sequence_gestures(df):
    seq_ids = df["sequence_id"].values
    seq_starts = np.array(df[df["sequence_counter"]==0].index)
    sequence_gestures = df.loc[seq_starts, gesture_columns]
    return sequence_gestures

In [ ]:
sequence_targets = get_sequence_targets(train_df)
print(len(sequence_targets[sequence_targets == 0]))
print(len(sequence_targets[sequence_targets == 1]))

In [ ]:
len_target = len(sequence_targets[sequence_targets == 1])
len_non_target = len(sequence_targets[sequence_targets == 0])
type_sample_weight = (len_target/len_non_target, len_target/len_target)

In [ ]:
type_sample_weight

In [ ]:
gestures.columns

In [ ]:
non_target_gestures = ["Drink from bottle/cup", "Glasses on/off", "Pull air toward your face",
                       "Pinch knee/leg skin", "Scratch knee/leg skin", "Write name on leg",
                       "Text on phone", "Feel around in tray and pull out an object",
                       "Write name in air", "Wave hello"]
sequence_gestures = get_sequence_gestures(train_df)
all_count = sequence_gestures.sum()
non_target_count = all_count[non_target_gestures]
non_target_sum = non_target_count.sum()
all_count[non_target_gestures] = non_target_sum
all_count = non_target_sum / all_count
gesture_sample_weight = tuple(all_count.values)

In [ ]:
gesture_sample_weight

In [ ]:
print(len(type_train_ds))
print(len(type_valid_ds))
print(len(gesture_train_ds))
print(len(gesture_valid_ds))

In [ ]:
# Check dataset shape
for i, (inputs, targets) in enumerate(type_train_ds):
    if i == 0:
        print(inputs[0].shape)        # Image (TOF)
        print(inputs[1].shape)        # Other time-series
        print(inputs[2].shape)        # Feature
        print(targets.shape)          # Target or Non-Target
        break

In [ ]:
for i, (inputs, targets) in enumerate(gesture_train_ds):
    if i == 0:
        print(inputs[0].shape)        # Image (TOF)
        print(inputs[1].shape)        # Other time-series
        print(inputs[2].shape)        # Feature
        print(targets.shape)          # Gesture
        break

In [ ]:
# test_ds = create_dataset(test_df, batch_size=64, mode="test")

In [ ]:
# Check dataset shape
# for inputs in test_ds:
#     print(inputs[0].shape)        # Image (TOF)
#     print(inputs[1].shape)        # Other time-series
#     print(inputs[2].shape)        # Feature

In [ ]:
import os
output_path = "/kaggle/working/"
type_train_path = os.path.join(output_path, "type_train_ds")
type_valid_path = os.path.join(output_path, "type_valid_ds")
gesture_train_path = os.path.join(output_path, "gesture_train_ds")
gesture_valid_path = os.path.join(output_path, "gesture_valid_ds")
# test_path = os.path.join(output_path, "test_ds")

tf.data.experimental.save(type_train_ds, type_train_path)
tf.data.experimental.save(type_valid_ds, type_valid_path)
tf.data.experimental.save(gesture_train_ds, gesture_train_path)
tf.data.experimental.save(gesture_valid_ds, gesture_valid_path)
# tf.data.experimental.save(test_ds, test_path)

In [ ]:
import json

mapping_dicts = {
    "type_mapping":type_mapping, 
    "gesture_mapping": gesture_mapping
}

dict_path = os.path.join(output_path, "mapping.json")
with open(dict_path, "w") as f:
    json.dump(mapping_dicts, f, indent=2)

In [ ]:
sample_weight_dicts = {
    "type_sample_weight": type_sample_weight,
    "gesture_sample_weight": gesture_sample_weight
}

dict_path = os.path.join(output_path, "sample_weight.json")
with open(dict_path, "w") as f:
    json.dump(sample_weight_dicts, f, indent=2)